In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import QuantileRegressor

# 1. Load & prepare data
df = pd.read_csv('cleaned_algae_data_complete.csv')

# Environmental feature columns
col_env = ['water_temperature', 'light', 'inorganic_nitrogen', 'total_phosphorus', 'secchi_depth']

# X = environment features, y = chlorophyll (biomass proxy)
X = df[col_env].values
y = df['chlorophyll'].values

# 2. Compute valid environmental ranges
mins = df[col_env].min().values
maxs = df[col_env].max().values

# 3. Fit a 95% Quantile Regressor
qr = QuantileRegressor(quantile=0.95, alpha=0.0)
qr.fit(X, y)

# 4. Constants
FRESH_TO_DRY_RATIO = 0.1               # 10:1 fresh to dry
DRY_TO_CHLORO_RATIO = 1 / 20           # 20:1 dry to chlorophyll (model is trained on chlorophyll)
C_PCT = 0.33
N_PCT = 0.025
P_PCT = 0.003

PRICE_C = 0.13    # per lb
PRICE_N = 5.08
PRICE_P = 11.15

# 5. Nutrient & Value Calculation
def calculate_phyco_credits(dry_biomass_lb):
    c_lb = dry_biomass_lb * C_PCT
    n_lb = dry_biomass_lb * N_PCT
    p_lb = dry_biomass_lb * P_PCT

    value_c = c_lb * PRICE_C
    value_n = n_lb * PRICE_N
    value_p = p_lb * PRICE_P
    total_value = value_c + value_n + value_p

    return {
        'carbon_lb': c_lb,
        'nitrogen_lb': n_lb,
        'phosphorus_lb': p_lb,
        'value_c_usd': value_c,
        'value_n_usd': value_n,
        'value_p_usd': value_p,
        'total_usd': total_value
    }

def assess_biomass_feasibility(env_vars, reported_biomass_lb, is_dry_input, qr_model):
    x = np.array(env_vars).reshape(1, -1)
    q95_chl = float(qr_model.predict(x))

    # Convert to dry biomass (if needed)
    dry_biomass = reported_biomass_lb if is_dry_input else reported_biomass_lb * FRESH_TO_DRY_RATIO
    q95_dry = q95_chl / DRY_TO_CHLORO_RATIO  # upper bound dry biomass
    feasible = dry_biomass <= q95_dry
    ratio = dry_biomass / q95_dry

    credit_info = calculate_phyco_credits(dry_biomass)

    return {
        'feasible': feasible,
        'q95_chlorophyll': q95_chl,
        'q95_dry_biomass_lb': q95_dry,
        'reported_input_biomass_lb': reported_biomass_lb,
        'input_type': 'dry' if is_dry_input else 'fresh',
        'reported_dry_biomass_lb': dry_biomass,
        'ratio': ratio,
        **credit_info
    }

def assess_with_domain_check(env_vars, reported_biomass_lb, is_dry_input, qr_model, mins, maxs):
    env = np.array(env_vars)
    if np.any(env < mins) or np.any(env > maxs):
        return {
            'feasible': False,
            'reason': 'environmental conditions outside training range'
        }
    return assess_biomass_feasibility(env_vars, reported_biomass_lb, is_dry_input, qr_model)



# %%
# 8. Example usage
env = [2, 14.2, 0.74, 0.24, 2.92]
reported_biomass_lb = 1500       # farmer input
is_dry_input = False            # ✅ True if dry, False if fresh

result = assess_with_domain_check(env, reported_biomass_lb, is_dry_input, qr, mins, maxs)


if result.get('feasible'):
    print(f"\n✅ Feasible biomass report!\n")
    print(f"Input type:              {result['input_type']} biomass")
    print(f"Reported biomass:        {result['reported_input_biomass_lb']:.2f} lb")
    print(f"Converted dry biomass:   {result['reported_dry_biomass_lb']:.2f} lb")
    print(f"Upper bound (dry):       {result['q95_dry_biomass_lb']:.2f} lb")
    print(f"Reported / Q95 ratio:    {result['ratio']:.2f}")

    print("\n🌿 Nutrient Removal (based on dry biomass):")
    print(f"• Carbon (33%):          {result['carbon_lb']:.2f} lb")
    print(f"• Nitrogen (2.5%):        {result['nitrogen_lb']:.2f} lb")
    print(f"• Phosphorus (0.3%):      {result['phosphorus_lb']:.2f} lb")

    print("\n💸 Credit Prices Used:")
    print("• Carbon:     $0.13 per lb")
    print("• Nitrogen:   $5.08 per lb")
    print("• Phosphorus: $11.15 per lb")

    print("\n🔢 PhycoCoin Calculation:")
    print("Total PhycoCoin value = dry_biomass_lb × (")
    print("  0.33 × 0.13 + 0.025 × 5.08 + 0.003 × 11.15 )")
    print(f"=> Total PhycoCoin value: ${result['total_usd']:.2f}")
else:
    print(f"\n❌ Infeasible biomass report:")
    print(result.get('reason', 'Reported biomass exceeds model threshold'))

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_algae_data_complete.csv'

In [4]:
import pickle

# Specify the filename to save the model
model_filename = 'quantile_regressor_model.pkl'

# Save the trained model to a file
with open(model_filename, 'wb') as f:
    pickle.dump(qr, f)

print(f"Model saved successfully to '{model_filename}'")

Model saved successfully to 'quantile_regressor_model.pkl'
